## Reddit Crawling

### 크롤링 막힌듯?

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd

In [ ]:
user_agent = user_agent = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"}

reddit = 'https://www.reddit.com/r/Music/'

tag = []
content = []
date = []

res = requests.get(reddit, user_agent)
soup = BeautifulSoup(res.text, "html.parser")
# driver = webdriver.Chrome()

# driver.get(reddit)
# time.sleep(1)

# page = soup.find("div", "rpBJOHq2PR60pnwJlUyP0")

titles = soup.find_all('h3')
for title in titles:
    print('titile: ', title.text)

time.sleep(1)

contents = soup.find_all(['p', 'li'])
for content in contents:
    print(content.text)

# search = driver.find_element(By.XPATH, '//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div[2]/div/div[2]/div/div/div/div[1]/div/div/div/form/div[1]/div/div/div/label/div[2]/div/input')
# ActionChains(driver).send_keys_to_element(search, "hip pop").perform(1)
# search.send_keys(Keys.RETURN)


Your request has been blocked due to a network policy.
Try logging in or creating an account here to get back to browsing.
If you're running a script or application, please register or sign in with your developer credentials here. Additionally make sure your User-Agent is not empty and is something unique and descriptive and try again. if you're supplying an alternate User-Agent string,
try changing back to default as that can sometimes result in a block.
You can read Reddit's Terms of Service here.
if you think that we've incorrectly blocked you or you would like to discuss
easier ways to get the data you want, please file a ticket here.
when contacting us, please include your ip address which is: 34.42.94.142 and reddit account


### reddit API 사용

In [ ]:
import praw

reddit = praw.Reddit(
    client_id='***',
    client_secret='***',
    user_agent='bokyungg',
    username='bokyungg',
    password='***'
)

In [ ]:
red = {}

def get_content_reddit(subred, keyword):
    subreddit = reddit.subreddit(subred)
    for submission in subreddit.hot(limit=100):
        id = submission.id
        title = submission.title
        content = submission.selftext
        if id not in red.keys():
            if keyword in content.lower():
                red[id] = {'title': title, 'content': content}
        if len(red) == 100:
            break
    return red

In [ ]:
get_content_reddit('Music', 'pop')

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



{'18tkp7o': {'title': 'In need of some new music',
  'content': 'I hope this kind of post is okay here. I feel like I’m a bit stuck when it comes to finding new music. I listen to 60s, 70s and 80s mainly. Classic rock, psychedelic rock, folk rock, new wave, some newer alternative and pop. I’d say I’m familiar with most mainstream bands in these genres and looking to branch out. Do not like metal or country. Thankful for any and all suggestions!!'},
 '18tewlk': {'title': 'Listened to MJs off the wall for the first time',
  'content': "I was a baby in 82.  I never heard the entirety of Michael Jackson's off the wall on the radio.   When it would come on when I was little, they would play off the wall, and possibly girlfriend and that's it.\n\nI bought the album off of iTunes and holy cow did I miss out on a lot.\n\nI forgot the radio plays a few tracks from a album or doesn't play obscure albums from popular artists.\n\nOff the wall is pretty underrated.  It's got this funk synth bass on

In [ ]:
red_df = pd.DataFrame(red)
red_df = red_df.transpose()

In [ ]:
red_df.head()

,title,content
18tkp7o,In need of some new music,I hope this kind of post is okay here. I feel ...
18tewlk,Listened to MJs off the wall for the first time,I was a baby in 82. I never heard the entiret...
18taj3d,To those who lived in the 90s in the US and Ca...,I feel like eurodance and it's influences can ...
18tkxam,Cherry Video - GEMINI DRIVE (electronic) - If ...,"Hey, \n\nNew to this and looking for like mind..."


### 글 내용 감정분석

- 간단하게 textblob으로 0~5 사이로 나타냄

In [ ]:
from textblob import TextBlob

def analyze_sentiment(red):
    analysis = TextBlob(red)

    # 0-5 범위로 변환
    sentiment_score = (analysis.sentiment.polarity + 1) * 2.5
    return sentiment_score

In [ ]:
red_df['sentiment_score'] = red_df['content'].apply(analyze_sentiment)
red_df.head()

,title,content,sentiment_score
18tkp7o,In need of some new music,I hope this kind of post is okay here. I feel ...,3.462730
18tewlk,Listened to MJs off the wall for the first time,I was a baby in 82. I never heard the entiret...,2.646528
18taj3d,To those who lived in the 90s in the US and Ca...,I feel like eurodance and it's influences can ...,2.950000
18tkxam,Cherry Video - GEMINI DRIVE (electronic) - If ...,"Hey, \n\nNew to this and looking for like mind...",2.879239


In [ ]:
red_df.to_parquet('reddit.parquet', index=False)